<a href="https://colab.research.google.com/github/haizzzi/lecture_notes/blob/main/Llama_3_8b_base_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) 런타임: GPU(T4) 선택
# 1) 필수 패키지 설치
!pip -q install -U transformers accelerate bitsandbytes sentencepiece



In [ ]:
# 2) (한번만) 허깅페이스 로그인
from huggingface_hub import login
login("hf_")  # 이러면 위젯 출력이 안 남음


In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 캐시 경로 설정
cache_dir = "/content/drive/MyDrive/huggingface_cache"
os.environ["HF_HOME"] = "/content/drive/MyDrive/huggingface_cache"   # 환경변수도 동일하게 지정
os.environ["HF_TOKEN"] = "hf_"   # 필요 시 Hugging Face 토큰

# 모델 ID
model_id = "meta-llama/Llama-3.1-8B"

# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",       # nf4 권장
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 토크나이저 로드
tok = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
    cache_dir=cache_dir
)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # bf16 권장
    cache_dir=cache_dir
)



In [ ]:
# 4) 간단 생성
prompt = "Hi. this is a new system of ai."
inputs = tok(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    out = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tok.eos_token_id
    )
print(tok.decode(out[0], skip_special_tokens=True))
